In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201030-1014-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [3]:
run_hash="e8a43dd1"
ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_ranked.ht')

In [4]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_

In [7]:
ht=ht.annotate(score=1-ht.rf_probability["TP"])

In [8]:
ht.score.show()

locus,alleles,score
locus<GRCh38>,array<str>,float64
chr1:12938,"[""GCAAA"",""G""]",5.89e-01
chr1:13024,"[""G"",""A""]",3.30e-01
chr1:13087,"[""A"",""G""]",2.79e-01
chr1:13116,"[""T"",""C""]",3.30e-01
chr1:13130,"[""C"",""T""]",3.30e-01
chr1:13151,"[""G"",""C""]",3.30e-01
chr1:13164,"[""G"",""A""]",3.30e-01
chr1:13176,"[""G"",""T""]",2.79e-01
chr1:13198,"[""C"",""A""]",7.01e-01


In [ ]:
e8a43dd1_rf_result_ranked_BINS.ht

In [9]:
ht.write(f'{tmp_dir}/{run_hash}/{run_hash}_rf_result_ranked_score.ht')

2020-10-30 10:19:57 Hail: INFO: wrote table with 7863547 rows in 500 partitions to hdfs://spark-master:9820//e8a43dd1/e8a43dd1_rf_result_ranked_score.ht


In [3]:
#de novo
ht_fam=hl.read_table(f"{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_trios_stats.ht")

In [4]:
ht_fam.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'n_transmitted_raw': int64 
    'n_untransmitted_raw': int64 
    'n_transmitted_adj': int64 
    'n_untransmitted_adj': int64 
    'n_de_novos_raw': int64 
    'n_de_novos_adj': int64 
    'ac_parents_raw': int64 
    'an_parents_raw': int64 
    'ac_children_raw': int64 
    'an_children_raw': int64 
    'ac_parents_adj': int64 
    'an_parents_adj': int64 
    'ac_children_adj': int64 
    'an_children_adj': int64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [11]:
from gnomad.utils.filtering import filter_to_adj

In [12]:
fam = "s3a://DDD-ELGH-UKBB-exomes/trios/DDD_trios.fam"
ped = hl.Pedigree.read(fam)
mt=mt_trios
de_novo_table = hl.de_novo(mt, ped, mt.family_stats[0].unrelated_qc_callstats.AF[1])
de_novo_table = de_novo_table.key_by('locus', 'alleles').collect_by_key('de_novo_data')

AttributeError: MatrixTable instance has no field, method, or property 'family_stats'
    Hint: use 'describe()' to show the names of all data fields.

In [6]:
mt_trios.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
    'proband': struct {
        s: str, 
        cohort: str, 
        f_stat: float64, 
        is_female: bool, 
        sex: str, 
        data_type: str, 
        original_pop: str, 
        known_pop: str, 
        superpopulation: str, 
        gVCF: str, 
        assigned_pop: str, 
        assigned_superpop: str, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
         

In [2]:
ht =hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/e8a43dd1/e8a43dd1_rf_result_ranked.ht')

In [3]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_

In [6]:
ht.biallelic_rank.show()

locus,alleles,biallelic_rank
locus<GRCh38>,array<str>,int64
chr1:12938,"[""GCAAA"",""G""]",61773
chr1:13024,"[""G"",""A""]",734811
chr1:13087,"[""A"",""G""]",310288
chr1:13116,"[""T"",""C""]",734812
chr1:13130,"[""C"",""T""]",734813
chr1:13151,"[""G"",""C""]",734814
chr1:13164,"[""G"",""A""]",734815
chr1:13176,"[""G"",""T""]",310289
chr1:13198,"[""C"",""A""]",2565396


In [4]:
ht.singleton_rank.summarize()

Non-missing,606969 (7.72%)
Missing,7256578 (92.28%)
Minimum,0
Maximum,573123
Mean,271526.22
Std Dev,172277.82


In [7]:
ht.singleton_rank.show()

locus,alleles,singleton_rank
locus<GRCh38>,array<str>,int64
chr1:12938,"[""GCAAA"",""G""]",NA
chr1:13024,"[""G"",""A""]",NA
chr1:13087,"[""A"",""G""]",NA
chr1:13116,"[""T"",""C""]",NA
chr1:13130,"[""C"",""T""]",NA
chr1:13151,"[""G"",""C""]",NA
chr1:13164,"[""G"",""A""]",NA
chr1:13176,"[""G"",""T""]",NA
chr1:13198,"[""C"",""A""]",NA


In [4]:
ht.show(5)

locus,alleles,biallelic_bin,singleton_bin
locus<GRCh38>,array<str>,int32,int32
chr1:12938,"[""GCAAA"",""G""]",31,NA
chr1:13024,"[""G"",""A""]",41,NA
chr1:13087,"[""A"",""G""]",12,NA
chr1:13116,"[""T"",""C""]",31,NA
chr1:13130,"[""C"",""T""]",22,NA


In [12]:
ht.tp.summarize()

Non-missing,81822 (1.04%)
Missing,7781725 (98.96%)
Counts,{True: 81822}


In [13]:
ht.fp.summarize()

Non-missing,7862337 (99.98%)
Missing,1210 (0.02%)
Counts,"{False: 7503511, True: 358826}"


In [14]:
ht.rf_train.summarize()

Non-missing,435515 (5.54%)
Missing,7428032 (94.46%)
Counts,"{False: 298100, True: 137415}"


In [15]:
ht.rf_label.summarize()

Non-missing,430382 (5.47%)
Missing,7433165 (94.53%)
Min Size,2
Max Size,2
Mean Size,2.00
Sample Values,"['FP', 'FP', 'FP', 'FP', 'FP']"


In [16]:
ht.rf_test.summarize()

Non-missing,7863547 (100.00%)
Missing,0
Counts,"{False: 7077605, True: 785942}"


In [23]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_table_for_RF_by_variant_type.ht')


In [24]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'feature_imputed': struct {
        a_index: bool, 
        n_alt_alleles: bool
    } 
----------------------------------------
Key: ['locus', 'alleles']
-

In [40]:
ht_training=hl.read_table('hdfs://spark-master:9820//models/6331642e/training.ht')

In [41]:
ht.omni.summarize()

Non-missing,64417 (0.82%)
Missing,7790190 (99.18%)
Counts,{True: 64417}


In [26]:
ht.mills.summarize()

Non-missing,17319 (0.22%)
Missing,7837288 (99.78%)
Counts,{True: 17319}


In [27]:
ht.fail_hard_filters.summarize()

Non-missing,7853399 (99.98%)
Missing,1208 (0.02%)
Counts,"{False: 7495017, True: 358382}"


In [28]:
ht.kgp_phase1_hc.summarize()

Non-missing,348055 (4.43%)
Missing,7506552 (95.57%)
Counts,{True: 348055}


In [29]:
ht.hapmap.summarize()

Non-missing,70005 (0.89%)
Missing,7784602 (99.11%)
Counts,{True: 70005}


In [34]:
fp_expr = ht.fail_hard_filters
tp_expr = ht.omni | ht.mills | ht.kgp_phase1_hc | ht.hapmap

In [35]:
fp_expr.summarize()

Non-missing,7853399 (99.98%)
Missing,1208 (0.02%)
Counts,"{False: 7495017, True: 358382}"


In [36]:
tp_expr.summarize()

Non-missing,371387 (4.73%)
Missing,7483220 (95.27%)
Counts,{True: 371387}


In [37]:
tp_expr = tp_expr | ht.transmitted_singleton

In [39]:
tp_expr.summarize()

Non-missing,956907 (12.18%)
Missing,6897700 (87.82%)
Counts,{True: 956907}


In [42]:
ht_training.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
  

In [45]:
ht_training.globals.show()

.feature_medians,.variants_by_strata,.features_importance,.features,.test_results
"dict<tuple(str), struct{a_index: int32, n_alt_alleles: int32}>","dict<tuple(str), int64>","dict<str, float64>",array<str>,"array<struct{rf_prediction: str, rf_label: str, n: int32}>"
"{(""indel""):(1,1)},(""snv""):(1,1)}}","{(""indel""):710415},(""snv""):7144192}}","{""variant_type"":2.97e-01},""allele_type"":1.72e-01},""has_star"":0.00e+00},""I...","[""InbreedingCoeff"",""variant_type"",""allele_type"",""n_alt_alleles"",""was_mixe...","[(""FP"",""FP"",14),(""FP"",""TP"",12),(""TP"",""FP"",69),(""TP"",""TP"",111)]"


In [43]:
ht_training.rf_train.summarize()

Non-missing,1297280 (16.50%)
Missing,6566267 (83.50%)
Counts,"{False: 618138, True: 679142}"


In [46]:
ht_training.rf_test.summarize()

Non-missing,7863547 (100.00%)
Missing,0
Counts,"{False: 7862062, True: 1485}"


In [47]:
ht_training.tp.summarize()

Non-missing,957941 (12.18%)
Missing,6905606 (87.82%)
Counts,{True: 957941}


In [48]:
ht_training.fp.summarize()

Non-missing,7862337 (99.98%)
Missing,1210 (0.02%)
Counts,"{False: 7503511, True: 358826}"


In [49]:
ht2=hl.read_table('hdfs://spark-master:9820//models/cfe6c371/training.ht')

In [50]:
ht2.fp.summarize()

Non-missing,7862337 (99.98%)
Missing,1210 (0.02%)
Counts,"{False: 7503511, True: 358826}"


In [51]:
ht2.tp.summarize()

Non-missing,371843 (4.73%)
Missing,7491704 (95.27%)
Counts,{True: 371843}


In [52]:
ht2.globals.show()

.feature_medians,.variants_by_strata,.features_importance,.features,.test_results
"dict<tuple(str), struct{a_index: int32, n_alt_alleles: int32}>","dict<tuple(str), int64>","dict<str, float64>",array<str>,"array<struct{rf_prediction: str, rf_label: str, n: int32}>"
"{(""indel""):(1,1)},(""snv""):(1,1)}}","{(""indel""):710415},(""snv""):7144192}}","{""variant_type"":1.38e-01},""allele_type"":7.98e-02},""has_star"":0.00e+00},""I...","[""InbreedingCoeff"",""variant_type"",""allele_type"",""n_alt_alleles"",""was_mixe...","[(""FP"",""FP"",24),(""FP"",""TP"",10),(""TP"",""FP"",64),(""TP"",""TP"",49)]"


In [54]:
ht3=hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/cfe6c371/rf_result_sanger_cohorts_new.ht')

In [55]:
ht3.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_

In [56]:
ht3.rf_probability.summarize()

Non-missing,7863547 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,15727094 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Sample Values,"['FP', 'TP', 'FP', 'TP', 'FP']"


In [67]:
ht3.aggregate(hl.agg.counter(ht3.rf_prediction))


{'FP': 4834670, 'TP': 3027836, None: 1041}

In [ ]:
'tp': bool 
    'fp': bool 
    'rf_train': bool 
    'rf_label': str 
    'rf_test': bool 
    'rf_probability': dict<str, float64> 
    'rf_prediction': str 

In [68]:
ht3.aggregate(hl.agg.counter(ht3.rf_train))


{False: 22190, True: 699524, None: 7141833}

In [69]:
ht3.aggregate(hl.agg.counter(ht3.rf_label))


{'FP': 349871, 'TP': 362888, None: 7150788}

In [2]:
ht4=hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/e8a43dd1/rf_result_sanger_cohorts_new.ht')

In [3]:
ht4.rf_probability.summarize()

Non-missing,7863547 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,15727094 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Sample Values,"['FP', 'TP', 'FP', 'TP', 'FP']"


In [4]:
ht4.aggregate(hl.agg.counter(ht4.rf_train))


{False: 23107, True: 698607, None: 7141833}

In [5]:
ht4.aggregate(hl.agg.counter(ht4.rf_label))

{'FP': 349871, 'TP': 362888, None: 7150788}

In [6]:
ht4.aggregate(hl.agg.counter(ht4.rf_prediction))

{'FP': 4849919, 'TP': 3012587, None: 1041}